In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q llama-index
!pip install -q gradio
!pip install einops
!pip install accelerate

In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
import torch

documents = SimpleDirectoryReader("/content/Data").load_data()

In [ ]:
from llama_index.prompts.prompts import SimpleInputPrompt
system_prompt = "As a Q&A assistant, your primary objective is to respond to inquiries with the utmost precision, concentrating on delivering accurate answers in alignment with the given instructions and context."
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=100,
    generate_kwargs={"temperature": 0.3, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="microsoft/phi-2",
    model_name="microsoft/phi-2",
    device_map="cuda",
    model_kwargs={"torch_dtype": torch.bfloat16}
)
from llama_index.embeddings import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

def predict(input, history):
  response = query_engine.query(input)
  return str(response)

In [ ]:

import gradio as gr

gr.ChatInterface(predict).launch(share=True)